In [1]:
# -*- coding: utf-8 -*-
# Importations
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import os
import json 
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import psycopg2 as pg
import scipy.special
import pandas_profiling
from pandas.plotting import lag_plot
from bokeh.io import output_file, show, output_notebook
from bokeh.layouts import gridplot
from bokeh.palettes import all_palettes
from bokeh.plotting import figure
from bokeh.models import Band, ColumnDataSource

/Users/martin_daniel/venvs/venv/lib/python2.7/site-packages/matplotlib/__init__.py:997: UserWarning: Bad val "TkAgg or echo backend : agg" on line #1
	"backend : TkAgg or echo backend : agg
"
	in file "/Users/martin_daniel/.matplotlib/matplotlibrc"
	Key backend: Unrecognized backend string "tkagg or echo backend : agg": valid strings are [u'pgf', u'ps', u'Qt4Agg', u'GTK', u'GTKAgg', u'nbAgg', u'agg', u'cairo', u'MacOSX', u'GTKCairo', u'Qt5Agg', u'template', u'WXAgg', u'TkAgg', u'GTK3Cairo', u'GTK3Agg', u'svg', u'WebAgg', u'pdf', u'gdk', u'WX']
  (val, error_details, msg))
/Users/martin_daniel/venvs/venv/lib/python2.7/site-packages/pandas_profiling/plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  Fil

In [2]:
fct_haemo = pd.read_csv('../data/model/fct_haemodynamic.csv',index_col=0)
fct_haemo['id_patient'] = fct_haemo['id_patient'].astype(int)
fct_haemo['dc'] = fct_haemo['dc'].astype(float)
fct_haemo['fc'] = fct_haemo['fc'].astype(float)
fct_haemo['papdia'] = fct_haemo['papdia'].astype(float)
fct_haemo['papmoy'] = fct_haemo['papmoy'].astype(float)
fct_haemo['papsys'] = fct_haemo['papsys'].astype(float)
fct_haemo['pasd'] = fct_haemo['pasd'].astype(float)
fct_haemo['pasm'] = fct_haemo['pasm'].astype(float)
fct_haemo['pass'] = fct_haemo['pass'].astype(float)
fct_haemo['pnid'] = fct_haemo['pnid'].astype(float)
fct_haemo['pnim'] = fct_haemo['pnim'].astype(float)
fct_haemo['pnis'] = fct_haemo['pnis'].astype(float)

IOError: File ../data/model/fct_haemodynamic.csv does not exist

In [3]:
pandas_profiling.ProfileReport(fct_haemo).to_file(outputfile="fct_haemo_profiling.html")

In [4]:
dim_patient = pd.read_csv('../data/dim_patient.csv')

In [48]:
df = pd.merge(fct_haemo, dim_patient[['id_patient','date_transplantation']], on='id_patient')
df['timestamp'] = df['date_transplantation'] + '-' +df['time']
df = df[df['time'] != '0']
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
#https://machinelearningmastery.com/time-series-data-visualization-with-python/

# Plotting time series for one patient

In [20]:
df_melt = pd.melt(df, id_vars=['id_patient','timestamp'], value_vars=['dc','fc','papdia','papmoy','papsys','pasd','pasm','pass','pnid','pnim','pnis'])

In [ ]:
def plot_haemodynamic(id_patient):
    print('Displaying haemodynamic measures for patient # {}'.format(id_patient))
    g = sns.FacetGrid(df_melt[df_melt_bornes['id_patient'] == id_patient], col="variable", col_wrap=4, sharey=False)
    g = g.map(plt.plot, "timestamp", "value")

In [ ]:
plot_haemodynamic(400)

In [ ]:
#TODO : récupérer les seuils avec ça et les plotter sur les time series https://github.com/dataforgoodfr/batch_5_transplant/blob/exploration/notebook/CBAC/first-insight.ipynb

# Time series features extraction (from Clément)

In [6]:
def features_cat(df , path = "features.json") : 
    
    rules = get_json(path)
    for r in rules.keys() :
        
        try : 
            
            cond_str = get_conditions(rules[r])
            q = 'df["{}"] = df["{}"].map(lambda x : {})'.format(r, rules[r]["name"], cond_str)
            exec(q)
        except : 
            print("could not create columns {}".format(rules[r]["name"]))
            pass
        
    return(df)

def get_conditions(conditions):
    
    cond = str()
    for i in conditions["conditions"].keys():
        
        cond = cond + "{} if   {}  else ".format('"' + conditions["conditions"][i]  + '"' , i )

    cond = cond + '"unknown"'
    return(cond)

def get_json(path = "features.json") : 
    
    f = json.load(open(path))
    
    return(f)

def features_cat(df , dataset_name, path = "features_haemodynamic.json") : 


    conditions = get_json(path)
    
    for i in conditions.keys():
        
        cond = get_conditions(conditions[i])    
        name_new_col = conditions[i]['name']
        col = i

        q = '{}["{}"] = {}["{}"].map(lambda x : {})'.format(dataset_name , name_new_col, dataset_name ,  col , cond)
    
        exec(q)
    

def get_conditions(conditions) :

    q1 = " 'normal' if x >= {}  and  x <= {} else  " .format(conditions['normalite'][0], conditions['normalite'][1])
    q2 = " 'anormal' if x > {} and x < {} else ".format(conditions['etendue'][0], conditions['normalite'][0])
    q3 = "'anormal' if x > {} and x < {} else ".format(conditions['normalite'][1], conditions['etendue'][1])
    q4 = " 'dirty' if x < {} and x > {} else ".format(conditions['etendue'][0], conditions['etendue'][1])
    q5 = "'null' if x == 0 else 'unkown'"
    
    
    
    cond = q1 + q2 + q3 + q4 + q5
    
    #print(cond)
    
    return(cond)

In [7]:
df_featurescat = features_cat(df, 'df', 'features_haemodynamic.json')

# Plotting time series with normality thresholds for one patient

In [11]:
def get_cond(path):
    cond = pd.read_json(path).transpose()
    
    cond['etendue_min'] = cond['etendue'].map(lambda x: x[0])
    cond['etendue_max'] = cond['etendue'].map(lambda x: x[1])
    cond['norm_min'] = cond['normalite'].map(lambda x: x[0])
    cond['norm_max'] = cond['normalite'].map(lambda x: x[1])
    
    return cond

In [12]:
output_notebook()

def plot_haemodynamic_norm(id_patient, path):
    
    cond = get_cond(path)
        
    df_patient = df[df['id_patient'] == id_patient]
    p_list = []
    j = 0

    for i in cond.index:
        p = figure(plot_width=200, plot_height=200, title=i)
        p.line(df_patient['timestamp'], cond.loc[i,'norm_min'], color='red', line_width=2)
        p.line(df_patient['timestamp'], cond.loc[i,'norm_max'], color='red', line_width=2)
        p.circle(df_patient['timestamp'], df_patient[i], color=all_palettes['PuBu'][3][0])
        p.xgrid[0].grid_line_color=None
        p.xaxis.major_label_text_font_size = '0pt'
        p.xaxis.major_tick_line_color = None  
        p.xaxis.minor_tick_line_color = None  
        p.ygrid[0].grid_line_alpha=0.5
        p_list.append(p)
        j+=1

    # make a grid
    grid = gridplot(p_list, ncols=4)

    # show the results
    show(grid)

Loading BokehJS ...

In [15]:
plot_haemodynamic_norm(338, 'features_haemodynamic.json')

In [77]:
cond = get_cond('features_haemodynamic.json')

df_melt_norm = df_melt.merge(cond[['norm_min','norm_max']],right_index=True, left_on='variable')
df_melt_norm['diff_value_norm_max'] = np.nan
df_melt_norm.loc[df_melt_norm['value'] > df_melt_norm['norm_max'],'diff_value_norm_max'] = df_melt_norm['value'] - df_melt_norm['norm_max']
df_melt_norm['diff_value_norm_min'] = np.nan
df_melt_norm.loc[(df_melt_norm['value'] != 0.0) & (df_melt_norm['value'] < df_melt_norm['norm_min']),'diff_value_norm_min'] = df_melt_norm['norm_min'] - df_melt_norm['value']

In [81]:
df_melt_norm[df_melt_norm['id_patient'] == 400].describe()

,id_patient,value,norm_min,norm_max,diff_value_norm_max,diff_value_norm_min
count,5093.0,5093.000000,5093.000000,5093.000000,1134.000000,135.000000
mean,400.0,37.171176,34.454545,62.727273,24.038801,7.533333
std,0.0,44.144005,26.731443,44.489485,19.177949,7.505023
min,400.0,-8.000000,2.000000,5.000000,1.000000,1.000000
25%,400.0,0.000000,14.000000,35.000000,9.000000,1.500000
50%,400.0,16.000000,20.000000,35.000000,20.000000,6.000000
75%,400.0,74.000000,60.000000,100.000000,37.000000,11.000000
max,400.0,284.000000,80.000000,140.000000,184.000000,48.000000


# Scatter plot of a time serie to investigate its stationarity

In [ ]:
pat = df[df['id_patient'] == 400]
pat = pat.drop(columns=['time','date_transplantation'])
pat['papmoy_shift'] = pat['papmoy'].shift(1)
lag_plot(pat['papmoy'])

# Make a time series stationary with x_t = log(x_t) - log(x_(t-1))

In [ ]:
lag_plot(np.log(pat['papmoy_shift']) - np.log(pat['papmoy']))

In [ ]:
pat['log_papmoy'] = np.log(pat['papmoy_shift']) - np.log(pat['papmoy'])    
pat.plot(y='log_papmoy',x='timestamp')

In [ ]:
#pat

In [ ]:
df.id_patient.unique()